In [41]:
from sentence_transformers import SentenceTransformer, models
import pickle
import jsonlines
import time
import datasets
from datasets import Features, Sequence, Value, load_dataset, Dataset, load_from_disk
import pandas as pd
import faiss

In [42]:
base_path = './'

base_out_path = base_path + 'corpus/'

collection_in_file = base_path + 'passages.pickle'

doc_id_to_index_out_file = base_out_path + 'doc_id_to_index.pickle'
doc_index_to_id_out_file = base_out_path + 'doc_index_to_id.pickle'
doc_embeddings_file = base_out_path + 'doc_reps.pickle'

dataset_path = base_out_path + 'marco_subsampled_dataset'
index_path = base_out_path + 'marco_faiss_index'

inference_batch_size = 512

In [43]:
def read_pickle(input_file):
    with open(input_file, 'rb') as f:
       out = pickle.load(f)
    f.close()
    return out

def write_pickle(output, output_file):
    with open(output_file, 'wb') as f:
       pickle.dump(output, f, protocol=pickle.HIGHEST_PROTOCOL)
    f.close()

In [4]:
'''
def write_id_maps_collection(collection_file, doc_list_file, id_to_index_out_file, index_to_id_out_file):
    in_file = jsonlines.open(collection_file, 'r')

    index_to_id_map = {}
    id_to_index_map = {}
    doc_list = []

    index = 0
    for obj in in_file:
        doc_id = obj["id"]
        doc_content = obj["contents"]
        doc_list.append(doc_content)

        index_to_id_map[index] = int(doc_id)
        id_to_index_map[int(doc_id)] = index

        index = index + 1

    write_pickle(doc_list, doc_list_file)
    write_pickle(id_to_index_map, id_to_index_out_file)
    write_pickle(index_to_id_map, index_to_id_out_file)
    
    return (doc_list, id_to_index_map, index_to_id_map)
'''

'\ndef write_id_maps_collection(collection_file, doc_list_file, id_to_index_out_file, index_to_id_out_file):\n    in_file = jsonlines.open(collection_file, \'r\')\n\n    index_to_id_map = {}\n    id_to_index_map = {}\n    doc_list = []\n\n    index = 0\n    for obj in in_file:\n        doc_id = obj["id"]\n        doc_content = obj["contents"]\n        doc_list.append(doc_content)\n\n        index_to_id_map[index] = int(doc_id)\n        id_to_index_map[int(doc_id)] = index\n\n        index = index + 1\n\n    write_pickle(doc_list, doc_list_file)\n    write_pickle(id_to_index_map, id_to_index_out_file)\n    write_pickle(index_to_id_map, index_to_id_out_file)\n    \n    return (doc_list, id_to_index_map, index_to_id_map)\n'

In [5]:
def generate_embeddings(doc_list_file, doc_embeddings_out_file, inference_batch_size):   
    starttime = time.time()
    
    sentbert_model = initialize_model()
    
    with open(doc_list_file, 'rb') as f:
       doc_list = pickle.load(f)
    f.close()
    
    total_docs = len(doc_list)
    print("Total #docs: ", total_docs)
    
    # Go over documents and form sb reps for documents.
    document_vectors = sentbert_model.encode(doc_list, show_progress_bar=True, batch_size=inference_batch_size)
    print('Encoded documents: {:}'.format(document_vectors.shape))
    
    ## To save in file
    with open(doc_embeddings_out_file, 'wb') as f:
       pickle.dump(document_vectors, f, protocol=pickle.HIGHEST_PROTOCOL)
    f.close()
    
    del doc_list
    del document_vectors
    
    endtime = time.time()
    print("Total time taken in hours: " + str((endtime-starttime)/3600))
    
def initialize_model():
    sentbert_model = SentenceTransformer('sentence-transformers/msmarco-distilbert-base-tas-b')
    return sentbert_model

In [6]:
# generate_embeddings(collection_in_file, doc_embeddings_file, inference_batch_size)

In [44]:
embs = read_pickle(doc_embeddings_file)

docs = read_pickle(collection_in_file)

In [45]:
embs_list = []
for i in range(embs.shape[0]):
    embs_list.append(embs[i, :])

In [49]:
chunked_corpus = {"text" : docs}
df = pd.DataFrame(chunked_corpus)
dataset = Dataset.from_pandas(df)
print(dataset)

dataset = dataset.add_column("embeddings", embs_list)
dataset

dataset.save_to_disk(dataset_path)

Dataset({
    features: ['text'],
    num_rows: 1550080
})


Saving the dataset (0/11 shards):   0%|          | 0/1550080 [00:00<?, ? examples/s]

In [50]:
index = faiss.IndexHNSWFlat(embs.shape[1], 10, faiss.METRIC_INNER_PRODUCT) 
# not clear what 2nd argument is, says number of nearest neighbors
dataset.add_faiss_index("embeddings", custom_index=index)

# And save the index
dataset.get_index("embeddings").save(index_path)

  0%|          | 0/1551 [00:00<?, ?it/s]

In [36]:
# dataset2 = load_from_disk(dataset_path)  # to reload the dataset
# index2 = dataset2.load_faiss_index("embeddings", index_path)  # to reload the index